In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')
df.drop(['RowNumber','CustomerId'],inplace=True,axis=1)

In [ ]:
X = df.iloc[:,1:10]
y=df.iloc[:,11]
gender= pd.get_dummies(df['Gender'],drop_first=True)
geo=pd.get_dummies(df['Geography'],drop_first=True)
X=pd.concat([X,geo,gender],axis=1)
X=X.drop(['Geography','Gender'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=100)

sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [ ]:
import keras
from keras import layers
from keras import callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

model = keras.Sequential([layers.Dense(300,input_shape=[10],activation='relu',kernel_initializer='he_normal'),
                          layers.Dropout(rate=0.3),
                          layers.Dense(256,activation='relu',kernel_initializer='he_normal'),
                          layers.Dropout(rate=0.3),
                          layers.Dense(128,activation='relu',kernel_initializer='he_normal'),
                          layers.Dense(1,activation='sigmoid',kernel_initializer='glorot_normal')
                         ])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
cc=callbacks.EarlyStopping(min_delta=0.001,patience=20,restore_best_weights=True)
hist= model.fit(X_train,y_train,validation_split=0.33,batch_size=128,epochs=500,callbacks=[cc])

histo= pd.DataFrame(hist.history)

In [ ]:
import plotly 
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.offline import init_notebook_mode
hist2 = histo.loc[:,['loss']]
lis= pd.DataFrame({'epoch':[int(i) for i in range(1,len(hist2)+1)]})
hist2=pd.concat([hist2,lis],axis=1)
fig = px.line(x=hist2['epoch'],y=hist2['loss'])
fig.show()

In [ ]:
from sklearn.metrics import accuracy_score

y_pred=model.predict(X_test)
y_pred=(y_pred>0.5)

score=accuracy_score(y_pred,y_test)
score